In [41]:
import Modules.SQcircuit_extensions as sq_ext
import SQcircuit as sq
import qutip as qt
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import importlib

%matplotlib ipympl

importlib.reload(sq_ext)
importlib.reload(sq)
np.set_printoptions(linewidth=500)

# Two qubits capacitively coupled

This notebook studies the circuit proposed by KIT. It's lumped element model is:
<img src='Other/Two qubits - antisymmetric C coupling.jpg' width='500'>


### Circuit parameters

In [4]:
# Fundamental constants
h    = 6.626e-34
GHz = 1e9
e0   = 1.602e-19
Φ_0 = h/(2*e0)

# Circuit parameters
Csh = 15
C   = 15
Cg  = 10
CJ  = 3
Lq  = 25
Lr  = 10
Δ   = 0.1
EJ  = 10.0
l = Lq*(Lq+4*Lr) - 4*Δ**2

Cc = 15

### Circuit elements and loops

In [14]:
# Initialize loops
loop_qubit = sq.Loop(0.5)
loop1_qubit_C_qubit= sq.Loop(0.5)
loop2_qubit_C_qubit = sq.Loop(0.5)

# Circuit components
C_01, C_02, C_12, L_03, L_31, L_23, JJ_12 = [[] for _ in range(7)]
for loop in [loop_qubit, loop1_qubit_C_qubit, loop2_qubit_C_qubit]:
    C_01 .append(sq.Capacitor(C,       'fF'))
    C_02 .append(sq.Capacitor(C,       'fF'))
    C_12 .append(sq.Capacitor(CJ+Csh,  'fF'))
    L_03 .append(sq.Inductor(Lr,       'nH'))
    L_31 .append(sq.Inductor(Lq/2 - Δ, 'nH',  loops=[loop]))
    L_23 .append(sq.Inductor(Lq/2 + Δ, 'nH',  loops=[loop]))
    JJ_12.append(sq.Junction(EJ,       'GHz', loops=[loop]))
C_24 = sq.Capacitor(Cc, 'fF')


### Single qubit

In [15]:
elements_qubit = {
    # qubit 1, nodes [0, 1, 2, 3]
    (0, 3): [L_03[0]],
    (0, 1): [C_01[0]],
    (0, 2): [C_02[0]],
    (3, 1): [L_31[0]],
    (1, 2): [C_12[0], JJ_12[0]],
    (2, 3): [L_23[0]],
}
qubit = sq.Circuit(elements_qubit)
qubit.description()

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [16]:
sq_ext.print_transformation(qubit)

Mode 1 = [  0.0   0.0   1.0]
Mode 2 = [  1.0   1.0   0.6]
Mode 3 = [ -0.9   1.0   0.0]


### Two uncoupled qubits

In [17]:
elements_qubit_qubit = {
    # qubit 1, nodes [0, 1, 2, 3]
    (0, 3): [L_03[1]],
    (0, 1): [C_01[1]],
    (0, 2): [C_02[1]],
    (3, 1): [L_31[1]],
    (1, 2): [C_12[1], JJ_12[1]],
    (2, 3): [L_23[1]],
    # qubit 2, nodes [0, 4, 5, 6]
    (0, 6): [L_03[2]],
    (0, 4): [C_01[2]],
    (0, 5): [C_02[2]],
    (6, 4): [L_31[2]],
    (4, 5): [C_12[2], JJ_12[2]],
    (5, 6): [L_23[2]],
}

qubit_qubit = sq.Circuit(elements_qubit_qubit)
qubit_qubit.description()

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [18]:
sq_ext.print_transformation(qubit_qubit)

Mode 1 = [  0.0   0.0   1.0   0.0   0.0   0.0]
Mode 2 = [  0.0   0.0   0.0   0.0   0.0   1.0]
Mode 3 = [  1.0   1.0   0.6   0.0   0.0   0.0]
Mode 4 = [  0.0   0.0   0.0   1.0   1.0   0.6]
Mode 5 = [ -0.9   1.0   0.0   0.0   0.0   0.0]
Mode 6 = [  0.0   0.0   0.0  -0.9   1.0   0.0]


In [26]:
qubit_qubit.set_trunc_nums([1, 1, 3, 3, 7, 7])
_ = qubit_qubit.diag(4)

### Two qubits capacitively coupled

In [27]:
elements_qubit_C_qubit = {
    # qubit 1, nodes [0, 1, 2, 3]
    (0, 3): [L_03[1]],
    (0, 1): [C_01[1]],
    (0, 2): [C_02[1]],
    (3, 1): [L_31[1]],
    (1, 2): [C_12[1], JJ_12[1]],
    (2, 3): [L_23[1]],
    # qubit 2, nodes [0, 4, 5, 6]
    (0, 6): [L_03[2]],
    (0, 4): [C_01[2]],
    (0, 5): [C_02[2]],
    (6, 4): [L_31[2]],
    (4, 5): [C_12[2], JJ_12[2]],
    (5, 6): [L_23[2]],
    # capacitive coupling
    (2, 4): [C_24]
}

qubit_C_qubit = sq.Circuit(elements_qubit_C_qubit)
qubit_C_qubit.description()

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [28]:
sq_ext.print_transformation(qubit_C_qubit)

Mode 1 = [  0.0   0.0   0.9   0.0   0.0   1.0]
Mode 2 = [ -0.0  -0.0  -1.0   0.0   0.0   0.9]
Mode 3 = [  1.0   1.0   0.6   1.0   1.0   0.6]
Mode 4 = [  1.0   0.1   0.3  -0.2  -1.0  -0.4]
Mode 5 = [ -0.9   0.9   0.0   0.9  -1.0  -0.0]
Mode 6 = [ -0.3  -1.0  -0.4   1.0   0.3   0.4]


In [29]:
qubit_C_qubit.set_trunc_nums([1, 1, 3, 3, 7, 7])
_ = qubit_C_qubit.diag(4)

# SWT

In [42]:
H_eff = sq_ext.H_eff_SWT(qubit_qubit, qubit_C_qubit)

In [43]:
sq_ext.decomposition_in_pauli_4x4(H_eff /GHz, 2, Print=True)

 (98.84-0j)	*	 I ⨂I 
 (-6.27-0j)	*	 I ⨂σz 
 (-0.26+0j)	*	 σx ⨂σx 
 (-0.13+0j)	*	 σx ⨂σy 
 (0.13+0j)	*	 σy ⨂σx 
 (-1.15+0j)	*	 σy ⨂σy 
 (-6.46+0j)	*	 σz ⨂I 
 (0.34+0j)	*	 σz ⨂σz 


array([[98.84-0.j,  0.  -0.j, -0.  +0.j, -6.27-0.j],
       [-0.  -0.j, -0.26+0.j, -0.13+0.j,  0.  +0.j],
       [ 0.  +0.j,  0.13+0.j, -1.15+0.j, -0.  -0.j],
       [-6.46+0.j, -0.  -0.j,  0.  +0.j,  0.34+0.j]])